In [111]:
import polars as pl 
import numpy as np 
import matplotlib.pyplot as plt

In [112]:
filename = "final.csv"
q = (pl.scan_csv(filename).select(pl.col("CorrelationType"), pl.col("Size"), pl.col("Algorithm"), pl.col("ObjectiveValue"), pl.col("ExecutionTime")))

df = q.collect()
df

CorrelationType,Size,Algorithm,ObjectiveValue,ExecutionTime
str,i64,str,f64,f64
"""correlated_sc1""",300,"""GRASP""",706.0,0.0212
"""correlated_sc1""",300,"""ILS-2""",666.0,0.0017
"""correlated_sc1""",300,"""ILS-10""",709.0,0.0027
"""correlated_sc1""",300,"""ILS-50""",695.0,0.0042
"""correlated_sc1""",300,"""ILS-100""",715.0,0.0064
…,…,…,…,…
"""not_correlated_sc4""",1000,"""ILS-100""",1360.0,0.2595
"""not_correlated_sc4""",1000,"""ILS-300""",1327.0,0.1472
"""not_correlated_sc4""",1000,"""SA""",1190.0,0.0094


In [113]:
df = (df.group_by(["CorrelationType", "Size","Algorithm"], maintain_order=True))
print(df)

In [114]:
result1 = df.agg([
    pl.col("ObjectiveValue").mean().alias("Mean"),
    pl.col("ObjectiveValue").max().alias("Best"),
    pl.col("ObjectiveValue").std().alias("Solution std"),
    (100 * (pl.col("ObjectiveValue").std() / pl.col("ObjectiveValue").mean())).alias("Coef. Var. Sol."),
    pl.col("ExecutionTime").mean().alias("Mean Time"),
    pl.col("ExecutionTime").std().alias("Time std"),
    (100 * (pl.col("ExecutionTime").std() / pl.col("ExecutionTime").mean())).alias("Coef. Var. Time")
])
result1

CorrelationType,Size,Algorithm,Mean,Best,Solution std,Coef. Var. Sol.,Mean Time,Time std,Coef. Var. Time
str,i64,str,f64,f64,f64,f64,f64,f64,f64
"""correlated_sc1""",300,"""GRASP""",699.53,731.0,15.851988,2.266091,0.016882,0.006165,36.519241
"""correlated_sc1""",300,"""ILS-2""",691.595,745.0,22.451981,3.246406,0.001854,0.000595,32.066757
"""correlated_sc1""",300,"""ILS-10""",710.825,748.0,15.562304,2.18933,0.004544,0.001592,35.038136
"""correlated_sc1""",300,"""ILS-50""",697.84,751.0,16.204237,2.322056,0.006884,0.002682,38.958434
"""correlated_sc1""",300,"""ILS-100""",698.5,734.0,16.103934,2.305502,0.006966,0.00293,42.054469
…,…,…,…,…,…,…,…,…,…
"""not_correlated_sc4""",1000,"""ILS-100""",1454.975,1622.0,83.753793,5.756373,0.2590565,0.103547,39.970801
"""not_correlated_sc4""",1000,"""ILS-300""",1455.56,1623.0,81.516385,5.600345,0.269917,0.100909,37.385192
"""not_correlated_sc4""",1000,"""SA""",1259.94,1518.0,114.313755,9.072952,0.011763,0.003386,28.783735


In [115]:
data_from_image = [
    # Scenario 1
    # NC
    {"Type": "NC", "n": 300, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 588.1, "Time": 0.01},
    {"Type": "NC", "n": 300, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 681.4, "Time": 3.86},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 468.7, "Time": 0.01},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 560.8, "Time": 11.95},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 365.0, "Time": 0.01},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 492.0, "Time": 19.85},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 368.2, "Time": 0.01},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 457.1, "Time": 32.62},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 300.4, "Time": 0.01},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 404.6, "Time": 45.30},
    # C
    {"Type": "C", "n": 300, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 696.9, "Time": 0.01},
    {"Type": "C", "n": 300, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 768.7, "Time": 2.59},
    {"Type": "C", "n": 500, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 621.3, "Time": 0.01},
    {"Type": "C", "n": 500, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 842.2, "Time": 18.96},
    {"Type": "C", "n": 700, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 511.2, "Time": 0.01},
    {"Type": "C", "n": 700, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 727.5, "Time": 43.53},
    {"Type": "C", "n": 800, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 475.7, "Time": 0.01},
    {"Type": "C", "n": 800, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 667.3, "Time": 54.60},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 396.7, "Time": 0.01},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 598.2, "Time": 73.40},
    # FC
    {"Type": "FC", "n": 300, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 667.7, "Time": 0.01},
    {"Type": "FC", "n": 300, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 751.9, "Time": 3.03},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 637.6, "Time": 0.01},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 819.2, "Time": 17.38},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 576.1, "Time": 0.01},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 742.7, "Time": 41.45},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 505.2, "Time": 0.01},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 706.0, "Time": 55.45},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "CG", "Sol": 468.4, "Time": 0.01},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 1", "Algorithm": "MA", "Sol": 641.9, "Time": 78.17},

    # Scenario 2
    # NC
    {"Type": "NC", "n": 300, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 247.2, "Time": 0.01},
    {"Type": "NC", "n": 300, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 299.1, "Time": 1.37},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 177.8, "Time": 0.01},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 226.7, "Time": 2.45},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 140.8, "Time": 0.01},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 186.3, "Time": 3.24},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 136.5, "Time": 0.01},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 174.3, "Time": 3.80},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 103.3, "Time": 0.01},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 144.0, "Time": 4.31},
    # C
    {"Type": "C", "n": 300, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 329.1, "Time": 0.01},
    {"Type": "C", "n": 300, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 443.9, "Time": 1.89},
    {"Type": "C", "n": 500, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 230.3, "Time": 0.01},
    {"Type": "C", "n": 500, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 346.5, "Time": 3.16},
    {"Type": "C", "n": 700, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 193.2, "Time": 0.01},
    {"Type": "C", "n": 700, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 273.7, "Time": 4.29},
    {"Type": "C", "n": 800, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 169.8, "Time": 0.01},
    {"Type": "C", "n": 800, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 249.8, "Time": 4.99},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 144.2, "Time": 0.01},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 213.7, "Time": 6.56},
    # FC
    {"Type": "FC", "n": 300, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 360.1, "Time": 0.01},
    {"Type": "FC", "n": 300, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 465.4, "Time": 2.21},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 320.8, "Time": 0.01},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 406.5, "Time": 4.61},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 270.6, "Time": 0.01},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 348.9, "Time": 5.25},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 261.0, "Time": 0.01},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 325.9, "Time": 7.09},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "CG", "Sol": 226.3, "Time": 0.01},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 2", "Algorithm": "MA", "Sol": 289.9, "Time": 11.30},
]
data_from_image_sc3_4 = [
    # Scenario 3
    # NC
    {"Type": "NC", "n": 300, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 953.0, "Time": 0.01},
    {"Type": "NC", "n": 300, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1027.7, "Time": 1.23},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1265.5, "Time": 0.01},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1386.8, "Time": 8.03},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1458.9, "Time": 0.02},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1676.0, "Time": 57.38},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1537.0, "Time": 0.02},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1813.1, "Time": 124.10},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1676.3, "Time": 0.04},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1949.8, "Time": 200.31},
    # C
    {"Type": "C", "n": 300, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 919.4, "Time": 0.01},
    {"Type": "C", "n": 300, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 960.2, "Time": 1.23},
    {"Type": "C", "n": 500, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1363.3, "Time": 0.01},
    {"Type": "C", "n": 500, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1439.4, "Time": 5.16},
    {"Type": "C", "n": 700, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1762.7, "Time": 0.02},
    {"Type": "C", "n": 700, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1859.0, "Time": 25.19},
    {"Type": "C", "n": 800, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1949.8, "Time": 0.03},
    {"Type": "C", "n": 800, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 2068.7, "Time": 30.53}, # Corrected typo from image (1.23 -> 30.53 based on visual position and magnitude)
    {"Type": "C", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 2234.7, "Time": 0.05},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 2426.4, "Time": 86.82},
    # FC
    {"Type": "FC", "n": 300, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 894.8, "Time": 0.01},
    {"Type": "FC", "n": 300, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 952.0, "Time": 1.54},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1343.4, "Time": 0.01},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1423.9, "Time": 6.12},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1741.1, "Time": 0.02},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 1843.9, "Time": 20.94},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 1941.5, "Time": 0.03},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 2054.0, "Time": 34.51},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "CG", "Sol": 2284.1, "Time": 0.05},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 3", "Algorithm": "MA", "Sol": 2433.1, "Time": 83.24},

    # Scenario 4
    # NC
    {"Type": "NC", "n": 300, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 829.3, "Time": 0.01},
    {"Type": "NC", "n": 300, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 903.0, "Time": 1.32},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1017.7, "Time": 0.01},
    {"Type": "NC", "n": 500, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1168.3, "Time": 13.01},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1191.8, "Time": 0.02},
    {"Type": "NC", "n": 700, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1401.9, "Time": 36.40},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1243.0, "Time": 0.02},
    {"Type": "NC", "n": 800, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1449.4, "Time": 55.01},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1280.1, "Time": 0.04},
    {"Type": "NC", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1513.2, "Time": 89.64},
    # C
    {"Type": "C", "n": 300, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 855.0, "Time": 0.01},
    {"Type": "C", "n": 300, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 896.4, "Time": 1.16},
    {"Type": "C", "n": 500, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1220.4, "Time": 0.01},
    {"Type": "C", "n": 500, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1308.1, "Time": 5.91},
    {"Type": "C", "n": 700, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1595.2, "Time": 0.02},
    {"Type": "C", "n": 700, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1726.4, "Time": 16.28},
    {"Type": "C", "n": 800, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1742.8, "Time": 0.02},
    {"Type": "C", "n": 800, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1879.3, "Time": 35.92},
    {"Type": "C", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1844.2, "Time": 0.04}, # Corrected typo from image (0.02 -> 0.04 based on column consistency)
    {"Type": "C", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 2150.4, "Time": 111.68},
    # FC
    {"Type": "FC", "n": 300, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 842.3, "Time": 0.01},
    {"Type": "FC", "n": 300, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 889.7, "Time": 1.23},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1250.7, "Time": 0.01},
    {"Type": "FC", "n": 500, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1313.6, "Time": 4.80},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1673.3, "Time": 0.02},
    {"Type": "FC", "n": 700, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1759.9, "Time": 15.40},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1805.3, "Time": 0.03},
    {"Type": "FC", "n": 800, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 1937.2, "Time": 23.37},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "CG", "Sol": 1971.4, "Time": 0.04},
    {"Type": "FC", "n": 1000, "Scenario": "Scenario 4", "Algorithm": "MA", "Sol": 2302.3, "Time": 77.22},
]

In [ ]:
df_image_raw = pl.DataFrame(data_from_image + data_from_image_sc3_4)
type_map = {
    "NC": "not_correlated",
    "C": "correlated",
    "FC": "fully_correlated"
}

scenario_map = {
    "Scenario 1": "_sc1",
    "Scenario 2": "_sc2",
    "Scenario 3": "_sc3",
    "Scenario 4": "_sc4"
}

df_image_transformed = df_image_raw.with_columns([

    pl.col("Type").replace(type_map).alias("correlation_base"),
    pl.col("Scenario").replace(scenario_map).alias("scenario_suffix")
]).with_columns(

    (pl.col("correlation_base") + pl.col("scenario_suffix")).alias("CorrelationType")
).select([
    pl.col("CorrelationType"),
    pl.col("n").alias("Size"),
    pl.col("Algorithm"),
    pl.col("Sol").alias("Mean"),
    pl.col("Time").alias("Mean Time")
])

print("Transformed DataFrame from image:")
print(df_image_transformed)

# You can check the schema
print("\nSchema of transformed DataFrame:")
print(df_image_transformed.schema)

Transformed DataFrame from image:
shape: (120, 5)
┌──────────────────────┬──────┬───────────┬────────┬───────────┐
│ CorrelationType      ┆ Size ┆ Algorithm ┆ Mean   ┆ Mean Time │
│ ---                  ┆ ---  ┆ ---       ┆ ---    ┆ ---       │
│ str                  ┆ i64  ┆ str       ┆ f64    ┆ f64       │
╞══════════════════════╪══════╪═══════════╪════════╪═══════════╡
│ not_correlated_sc1   ┆ 300  ┆ CG        ┆ 588.1  ┆ 0.01      │
│ not_correlated_sc1   ┆ 300  ┆ MA        ┆ 681.4  ┆ 3.86      │
│ not_correlated_sc1   ┆ 500  ┆ CG        ┆ 468.7  ┆ 0.01      │
│ not_correlated_sc1   ┆ 500  ┆ MA        ┆ 560.8  ┆ 11.95     │
│ not_correlated_sc1   ┆ 700  ┆ CG        ┆ 365.0  ┆ 0.01      │
│ …                    ┆ …    ┆ …         ┆ …      ┆ …         │
│ fully_correlated_sc4 ┆ 700  ┆ MA        ┆ 1759.9 ┆ 15.4      │
│ fully_correlated_sc4 ┆ 800  ┆ CG        ┆ 1805.3 ┆ 0.03      │
│ fully_correlated_sc4 ┆ 800  ┆ MA        ┆ 1937.2 ┆ 23.37     │
│ fully_correlated_sc4 ┆ 1000 ┆ CG      

In [117]:
merged_df = pl.concat([result1, df_image_transformed], how='align')
merged_df

CorrelationType,Size,Algorithm,Mean,Best,Solution std,Coef. Var. Sol.,Mean Time,Time std,Coef. Var. Time
str,i64,str,f64,f64,f64,f64,f64,f64,f64
"""correlated_sc1""",300,"""CG""",696.9,null,null,null,0.01,null,null
"""correlated_sc1""",300,"""GRASP""",699.53,731.0,15.851988,2.266091,0.016882,0.006165,36.519241
"""correlated_sc1""",300,"""ILS-10""",710.825,748.0,15.562304,2.18933,0.004544,0.001592,35.038136
"""correlated_sc1""",300,"""ILS-100""",698.5,734.0,16.103934,2.305502,0.006966,0.00293,42.054469
"""correlated_sc1""",300,"""ILS-2""",691.595,745.0,22.451981,3.246406,0.001854,0.000595,32.066757
…,…,…,…,…,…,…,…,…,…
"""not_correlated_sc4""",1000,"""ILS-50""",1472.27,1664.0,86.802769,5.895846,0.197512,0.072046,36.476617
"""not_correlated_sc4""",1000,"""MA""",1513.2,null,null,null,89.64,null,null
"""not_correlated_sc4""",1000,"""REACT_GRASP""",1366.335,1561.0,83.754024,6.129831,0.284735,0.095013,33.368821


In [ ]:
merged_df = merged_df.with_columns(
    pl.col("Mean").filter(pl.col("Algorithm") == "CG").first().over(["CorrelationType", "Size"]).alias("Mean_CG_in_Group")
).with_columns(
    (
        100*(pl.col("Mean_CG_in_Group") - pl.col("Mean")) / pl.col("Mean_CG_in_Group")
    ).alias("Relative_Improvement_vs_CG")
)

In [ ]:
df_with_new_col = merged_df.with_columns(

    pl.col("Mean").filter(pl.col("Algorithm") == "MA").first().over(["CorrelationType", "Size"]).alias("Mean_MA_in_Group")
).with_columns(
    (
        100*(pl.col("Mean_MA_in_Group") - pl.col("Mean")) / pl.col("Mean_MA_in_Group")
    ).alias("Relative_Improvement_vs_MA")
)

In [120]:
df = df_with_new_col.drop(["Mean_MA_in_Group","Mean_CG_in_Group"])

In [ ]:
output_filename = "final_transformed.csv"
df.write_csv(output_filename)

In [ ]:
df_grasp = df.filter(pl.col("Algorithm").str.contains("GRASP"))
df_grasp = df_grasp.group_by(["Algorithm"], maintain_order=True)
df_grasp = df_grasp.agg([
    pl.col("Mean").mean().alias("Mean solution"),
    pl.col("Mean Time").mean().alias("Mean Time")
])
df_grasp

Algorithm,Mean solution,Mean Time
str,f64,f64
"""GRASP""",912.688935,0.094513
"""REACT_GRASP""",886.207546,0.074731


In [ ]:
df_ils = df.filter(pl.col("Algorithm").str.contains("ILS"))
df_ils = df_ils.group_by(["Algorithm"], maintain_order=True)
df_ils = df_ils.agg([
    pl.col("Mean").mean().alias("Mean solution"),
    pl.col("Mean Time").mean().alias("Mean Time")
])
df_ils

Algorithm,Mean solution,Mean Time
str,f64,f64
"""ILS-10""",958.738278,0.024057
"""ILS-100""",942.88387,0.058301
"""ILS-2""",907.876861,0.008857
"""ILS-300""",941.352611,0.058973
"""ILS-50""",949.838176,0.048255
